In [160]:
# загружаем необходимые библиотеки. BeautifulSoup — библиотека, которая позволяет работать с HTML- и XML-кодом.

from bs4 import BeautifulSoup 
import requests
import pandas as pd

In [161]:
# сохраняем в переменную URL-адрес страницы, с которой мы планируем парсить информацию. Обращаемся к сайту

url = 'https://ru.investing.com/equities/russia' 
r = requests.get(url)

In [162]:
# с помощью библиотеки BeautifulSoup преобразует сложный HTML-документ в сложное дерево объектов Python

soup = BeautifulSoup(r.text, 'lxml')

In [173]:
# ищем в исходном коде сайта Investing место, где хранится ссылка на страничку первой акции
# с помощью функции find "спускаемся" по тегам и достаем нужный кусок кода
# создаем требуемые переменные

stock_link = 'https://ru.investing.com' + soup.find('tbody', class_='datatable_body__cs8vJ').find('div', class_='datatable_cell__wrapper__ddKL4').find('a', class_='inv-link bold datatable_cell--name__link__tmnQz').get('href')
stock_name_rus = soup.find('tbody', class_='datatable_body__cs8vJ').find('div', class_='datatable_cell__wrapper__ddKL4').find('a', class_='inv-link bold datatable_cell--name__link__tmnQz').find('span', class_='pt-2 font-normal dynamic-table_cell-symbol__wp4Vw').text
ticker = soup.find('tbody', class_='datatable_body__cs8vJ').find('div', class_='datatable_cell__wrapper__ddKL4').find('a', class_='inv-link bold datatable_cell--name__link__tmnQz').find('span', class_='block text-ellipsis overflow-hidden whitespace-nowrap').text
stock_price = soup.find('tbody', class_='datatable_body__cs8vJ').find('td', class_='datatable_cell__0y0eu datatable_cell--align-end__fwomz dynamic-table_col-other__tKGpI').text
price_change = soup.find('tbody', class_='datatable_body__cs8vJ').find('td', class_='datatable_cell__0y0eu datatable_cell--align-end__fwomz datatable_cell--down__d61fR datatable_cell--bold__UPfqK dynamic-table_col-other__tKGpI').text
day_volume = soup.find('tbody', class_='datatable_body__cs8vJ').findAll('td', class_='datatable_cell__0y0eu datatable_cell--align-end__fwomz dynamic-table_col-other__tKGpI')[3].text
time = soup.find('tbody', class_='datatable_body__cs8vJ').find('span', class_='flex flex-row items-center').text

In [174]:
# считаем процент изменения цены

price_change_percent = str(round(float(price_change.replace('.','').replace(',','.'))/(float(stock_price.replace('.','').replace(',','.'))+float(price_change.replace('.','').replace(',','.')))*100, 2)) + '%'
price_change_percent

'-2.49%'

In [175]:
# создаем список акций, для которых будем доставать с сайта необходимые значения переменных 

stocks = soup.find('tbody', class_='datatable_body__cs8vJ').findAll('tr', class_='datatable_row__qHMpQ')

In [179]:
# создаем пустой список data, куда будем в цикле сохранять список наших переменных 

data = []
for stock in stocks:
    stock_link = 'https://ru.investing.com' + stock.find('div', class_='datatable_cell__wrapper__ddKL4').find('a', class_='inv-link bold datatable_cell--name__link__tmnQz').get('href')
    stock_name_rus = stock.find('div', class_='datatable_cell__wrapper__ddKL4').find('a', class_='inv-link bold datatable_cell--name__link__tmnQz').find('span', class_='pt-2 font-normal dynamic-table_cell-symbol__wp4Vw').text
    ticker = stock.find('div', class_='datatable_cell__wrapper__ddKL4').find('a', class_='inv-link bold datatable_cell--name__link__tmnQz').find('span', class_='block text-ellipsis overflow-hidden whitespace-nowrap').text
    stock_price = stock.find('td', class_='datatable_cell__0y0eu datatable_cell--align-end__fwomz dynamic-table_col-other__tKGpI').text
    try:
        price_change = stock.find('td', class_='datatable_cell__0y0eu datatable_cell--align-end__fwomz datatable_cell--down__d61fR datatable_cell--bold__UPfqK dynamic-table_col-other__tKGpI').text
    except:
        price_change = '0,0'
    price_change_percent = str(round(float(price_change.replace('.','').replace(',','.'))/(float(stock_price.replace('.','').replace(',','.'))+float(price_change.replace('.','').replace(',','.')))*100, 2)) + '%'
    day_volume = stock.findAll('td', class_='datatable_cell__0y0eu datatable_cell--align-end__fwomz dynamic-table_col-other__tKGpI')[3].text
    time = stock.find('span', class_='flex flex-row items-center').text
    
    data.append([stock_link, stock_name_rus, ticker, stock_price, price_change, price_change_percent, day_volume, time])

In [180]:
# создадим список с названиями колонок для нашего dataframe

header = ['stock_link', 'stock_name_rus', 'ticker', 'stock_price', 'price_change', 'price_change_percent', 'day_volume', 'time']

In [182]:
# создадим dataframe из списка списков data с названиями колонок из списка header

df = pd.DataFrame(data, columns=header)

In [183]:
# сохраним наш dataframe в файле .csv

df.to_csv('/Users/ch199/investing_data.csv', sep=';', encoding='utf-8')

In [184]:
# проверим, корректно ли отображается наш dataframe

df

,stock_link,stock_name_rus,ticker,stock_price,price_change,price_change_percent,day_volume,time
0,https://ru.investing.com/equities/afk-sistema_rts,Система,AFKS,"16,04","-0,39",-2.49%,"46,44M",17:09:14
1,https://ru.investing.com/equities/aeroflot,Аэрофлот,AFLT,"38,94","0,0",0.0%,"17,61M",17:09:36
2,https://ru.investing.com/equities/gmk-noril-ni...,Норникель,GMKN,"14.730,0","-6,0",-0.04%,"56,45K",17:09:26
3,https://ru.investing.com/equities/gazprom_rts,Газпром,GAZP,"173,89","-1,37",-0.79%,"22,56M",17:09:36
4,https://ru.investing.com/equities/lukoil_rts,ЛУКОЙЛ,LKOH,"5.039,5","-35,5",-0.71%,"501,8K",17:09:35
5,https://ru.investing.com/equities/mmk_rts,ММК,MAGN,"39,15","-0,28",-0.72%,"5,7M",17:09:12
6,https://ru.investing.com/equities/mts_rts,МТС,MTSS,"307,60","-5,95",-1.97%,"6,59M",17:08:55
7,https://ru.investing.com/equities/magnit_rts,Магнит,MGNT,"4.635,0","0,0",0.0%,"115,33K",17:08:57
8,https://ru.investing.com/equities/nlmk_rts,НЛМК,NLMK,"134,70","-0,08",-0.06%,"1,75M",17:09:15
9,https://ru.investing.com/equities/novatek_rts,НОВАТЭК,NVTK,"1.271,40","-6,80",-0.54%,"446,61K",17:09:14
